<a href="https://colab.research.google.com/github/shubhsudan/NLP_PROJECT/blob/main/NLP_AUTO_TEXT_GEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Directory containing the text files
directory_train = '/content/drive/MyDrive/hinglish/train'  # Replace with the actual directory path

# Output file
concat_train = '/content/drive/MyDrive/hinglish/final_training'

# List all files in the directory
files_to_concatenate = [os.path.join(directory_train, filename) for filename in os.listdir(directory_train) if filename.endswith('.txt')]

# Concatenate the files
with open(concat_train, 'w') as output:
    for file in files_to_concatenate:
        with open(file, 'r') as input:
            output.write(input.read())

print("Files concatenated to", concat_train)


Files concatenated to /content/drive/MyDrive/hinglish/final_training


WE HAVE USED BART (BIDIRECTIONAL AND AUTOREGRESSIVE TRANSFORMER)

In [ ]:
# Directory containing the text files
directory_validation = '/content/drive/MyDrive/hinglish/valid'  # Replace with the actual directory path

# Output file
concat_validation = '/content/drive/MyDrive/hinglish/concat_valid'

# List all files in the directory
files_to_concatenate = [os.path.join(directory_validation, filename) for filename in os.listdir(directory_validation) if filename.endswith('.txt')]

# Concatenate the files
with open(concat_validation, 'w') as output:
    for file in files_to_concatenate:
        with open(file, 'r') as input:
            output.write(input.read())

print("Files concatenated to", concat_validation)

Files concatenated to /content/drive/MyDrive/hinglish/concat_valid


In [ ]:
!pip install 'transformers[torch]'
!pip install 'accelerate'



In [ ]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ieq315rv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ieq315rv
  Resolved https://github.com/huggingface/transformers to commit 7d8ff3629b2725ec43ace99c1a6e87ac1978d433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Define training and validation dataloaders (replace with your actual dataloaders)
train_df = concat_train
val_df = concat_validation


In [ ]:
import shutil

# Remove the existing output directory
shutil.rmtree(concat_train, ignore_errors=True)

In [ ]:
import os
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, TrainingArguments, Trainer

# Initialize the BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Tokenize your training data
with open(train_df, "r", encoding="utf-8") as file:
    train_data = [line.strip() for line in file]

# Prepare the input data
input_data = tokenizer(train_df, truncation=True, padding=True, return_tensors="pt")

# Create a PyTorch dataset
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = TextDataset(input_data)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/hinglish/model",  # Specify the output directory
    num_train_epochs=500,  # Adjust the number of training epochs as needed
    per_device_train_batch_size=32,
    save_steps=10_00,
    learning_rate = 0.01,# Adjust the save frequency as needed
    save_total_limit=2,
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator= None,  # Default collator works for text generation tasks
    train_dataset=train_dataset,
)

# Start fine-tuning
#trainer.train()

# Save the model
model.save_pretrained("/content/drive/MyDrive/hinglish/final_model")
tokenizer.save_pretrained("/content/drive/MyDrive/hinglish/final_model")


('/content/drive/MyDrive/hinglish/final_model/tokenizer_config.json',
 '/content/drive/MyDrive/hinglish/final_model/special_tokens_map.json',
 '/content/drive/MyDrive/hinglish/final_model/vocab.json',
 '/content/drive/MyDrive/hinglish/final_model/merges.txt',
 '/content/drive/MyDrive/hinglish/final_model/added_tokens.json')

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/hinglish/final_model")
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/hinglish/final_model")

# Get user input
user_input = input("Enter a text prompt: ")

# Tokenize the user input
input_ids = tokenizer.encode(user_input, return_tensors="pt", max_length=1024, truncation=True)

# Generate text with adjusted parameters
output = model.generate(
    input_ids,
    max_length=30,  # Adjust the maximum length as needed
    min_length=10,   # Adjust the minimum length as needed
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)



Enter a text prompt: hey brother


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Generated Text:
hey brother. Hey brother, I've got a confession to make.


In [ ]:
!pip install tabulate

from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
from nltk import ngrams
from tabulate import tabulate

# Sample reference texts and generated texts (replace with your data)
reference_texts = ["i want to make a biopic on Real Heroes and not on Reel Hero", "And I'd be stupid if i made a biopic on myself"]
generated_texts = ["students are a very important part of  the future of our country, says President Obama.", "shubh how are you"]

# Calculate BLEU score
bleu_scores = [sentence_bleu([ref.split()], gen.split(), smoothing_function=SmoothingFunction().method1) for ref, gen in zip(reference_texts, generated_texts)]
corpus_bleu_score = corpus_bleu([[ref.split()] for ref in reference_texts], [gen.split() for gen in generated_texts], smoothing_function=SmoothingFunction().method1)

# Function to calculate ROUGE scores
def rouge_n(reference, generated, n):
    reference_ngrams = set(ngrams(reference.split(), n))
    generated_ngrams = set(ngrams(generated.split(), n))
    intersection = len(reference_ngrams.intersection(generated_ngrams))
    return intersection / len(reference_ngrams)

# Calculate ROUGE scores for each pair
rouge1_scores = [rouge_n(ref, gen, 1) for ref, gen in zip(reference_texts, generated_texts)]
rouge2_scores = [rouge_n(ref, gen, 2) for ref, gen in zip(reference_texts, generated_texts)]
rougeL_scores = [rouge_n(ref, gen, len(ref.split())) for ref, gen in zip(reference_texts, generated_texts)]

# Prepare the data for tabulation
table_data = []
for i, (ref, gen) in enumerate(zip(reference_texts, generated_texts)):
    table_data.append([f"Reference {i+1}", "Generated", bleu_scores[i], rouge1_scores[i], rouge2_scores[i], rougeL_scores[i]])
table_data.append(["Corpus", "Corpus", corpus_bleu_score, "-", "-", "-"])

# Create and display the table
headers = ["Reference", "Generated", "BLEU", "ROUGE-1", "ROUGE-2", "ROUGE-L"]
table = tabulate(table_data, headers, tablefmt="grid")
print(table)


+-------------+-------------+-----------+---------------------+-----------+-----------+
| Reference   | Generated   |      BLEU | ROUGE-1             | ROUGE-2   | ROUGE-L   |
+=============+=============+===========+=====================+===========+===========+
| Reference 1 | Generated   | 0.0132179 | 0.07692307692307693 | 0.0       | 0.0       |
+-------------+-------------+-----------+---------------------+-----------+-----------+
| Reference 2 | Generated   | 0         | 0.0                 | 0.0       | 0.0       |
+-------------+-------------+-----------+---------------------+-----------+-----------+
| Corpus      | Corpus      | 0.0081855 | -                   | -         | -         |
+-------------+-------------+-----------+---------------------+-----------+-----------+


In [ ]:
#BLEAU SCORE CODE
!pip install tabulate
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
from tabulate import tabulate

# Sample reference texts and generated texts (replace with your data)
reference_texts = ["i want to make a biopic on Real Heroes and not on Reel Hero", "And I'd be stupid if i made a biopic on myself"]
generated_texts = ["students are a very important part of  the future of our country, says President Obama.", "shubh how are you"]


# Calculate BLEU score
bleu_scores = [sentence_bleu([ref.split()], gen.split(), smoothing_function=SmoothingFunction().method1) for ref, gen in zip(reference_texts, generated_texts)]
corpus_bleu_score = corpus_bleu([[ref.split()] for ref in reference_texts], [gen.split() for gen in generated_texts], smoothing_function=SmoothingFunction().method1)

# Prepare the data for tabulation
table_data = []
for i, (ref, gen) in enumerate(zip(reference_texts, generated_texts)):
    table_data.append([f"Reference {i+1}", "Generated", bleu_scores[i]])
table_data.append(["Corpus", "Corpus", corpus_bleu_score])

# Create and display the table
headers = ["Reference", "Generated", "BLEU"]
table = tabulate(table_data, headers, tablefmt="grid")
print(table)


+-------------+-------------+-----------+
| Reference   | Generated   |      BLEU |
+=============+=============+===========+
| Reference 1 | Generated   | 0.0132179 |
+-------------+-------------+-----------+
| Reference 2 | Generated   | 0         |
+-------------+-------------+-----------+
| Corpus      | Corpus      | 0.0081855 |
+-------------+-------------+-----------+


In [ ]:
from nltk import ngrams
from tabulate import tabulate

# Sample reference texts and generated texts (replace with your data)
reference_texts = ["i want to make a biopic on Real Heroes and not on Reel Hero", "And I'd be stupid if i made a biopic on myself"]
generated_texts = ["students are a very important part of  the future of our country, says President Obama.", "shubh how are you"]



# Function to calculate ROUGE scores
def rouge_n(reference, generated, n):
    reference_ngrams = set(ngrams(reference.split(), n))
    generated_ngrams = set(ngrams(generated.split(), n))
    intersection = len(reference_ngrams.intersection(generated_ngrams))
    return intersection / len(reference_ngrams)

# Calculate ROUGE scores for each pair
rouge1_scores = [rouge_n(ref, gen, 1) for ref, gen in zip(reference_texts, generated_texts)]
rouge2_scores = [rouge_n(ref, gen, 2) for ref, gen in zip(reference_texts, generated_texts)]
rougeL_scores = [rouge_n(ref, gen, len(ref.split())) for ref, gen in zip(reference_texts, generated_texts)]

# Prepare the data for tabulation
table_data = []
for i, (ref, gen) in enumerate(zip(reference_texts, generated_texts)):
    table_data.append([f"Reference {i+1}", "Generated", rouge1_scores[i], rouge2_scores[i], rougeL_scores[i]])


# Create and display the table
headers = ["Reference", "Generated", "ROUGE-1", "ROUGE-2", "ROUGE-L"]
table = tabulate(table_data, headers, tablefmt="grid")
print(table)


+-------------+-------------+-----------+-----------+-----------+
| Reference   | Generated   |   ROUGE-1 |   ROUGE-2 |   ROUGE-L |
+=============+=============+===========+===========+===========+
| Reference 1 | Generated   | 0.0769231 |         0 |         0 |
+-------------+-------------+-----------+-----------+-----------+
| Reference 2 | Generated   | 0         |         0 |         0 |
+-------------+-------------+-----------+-----------+-----------+


In [ ]:
!pip install wordcloud


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


In [ ]:
!pip install flask

In [ ]:
!pip install jupyter-dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.9 MB/s eta 0:00:00


In [ ]:

import plotly.express as px
from jupyter_dash import JupyterDash

import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data


<ipython-input-7-eed5ee1d69eb>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-7-eed5ee1d69eb>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [1]:
!pip install transformers
import dash
from dash import dcc, html
import plotly.express as px
from dash.dependencies import Input, Output, State
from transformers import BartForConditionalGeneration, BartTokenizer

# Initialize your text generation model and tokenizer
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/hinglish/final_model")
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/hinglish/final_model")

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of your HTML template
app.layout = html.Div([
    html.H1("Text Generation App"),
    dcc.Textarea(id='user-input', placeholder='Enter a text prompt...'),
    html.Button('Generate Text', id='generate-button'),
    html.Div(id='generated-text'),
])

# Create a callback to generate text
@app.callback(
    Output('generated-text', 'children'),
    Input('generate-button', 'n_clicks'),
    State('user-input', 'value')
)
def generate_text(n_clicks, user_input):
    if n_clicks and user_input:
        input_ids = tokenizer.encode(user_input, return_tensors="pt", max_length=1024, truncation=True)
        output = model.generate(input_ids, max_length=150, num_return_sequences=1)
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        return generated_text
    else:
        return ''

if __name__ == '__main__':
    app.run_server(mode='external')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.2 MB/s eta 0:00:00


ModuleNotFoundError: ignored

In [2]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [18]:
%%writefile app.py
import os
# Specify the location of the requirements file
requirements_file = '/content/requirements.txt'

# Install the dependencies from the requirements file
!pip install -r $requirements_file


!pip install streamlit
!pip install transformers
import streamlit as st
from transformers import BartForConditionalGeneration, BartTokenizer

# Initialize your text generation model and tokenizer
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/hinglish/final_model")
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/hinglish/final_model")

# Define the Streamlit app
st.title("Text Generation App")
user_input = st.text_area("Enter a text prompt", "")
generate_button = st.button("Generate Text")

if generate_button and user_input:
    input_ids = tokenizer.encode(user_input, return_tensors="pt", max_length=1024, truncation=True)
    output = model.generate(input_ids, max_length=150, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    st.subheader("Generated Text:")
    st.write(generated_text)

st.write("")

# Optionally, add more features and descriptions to your app


Overwriting app.py
